In [47]:
import requests
import numpy as np
import pandas as pd
import sqlite3
import time
from tqdm import tqdm
import os
from random import randint
import string

In [48]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [49]:
def internet_connection():
    try:
        response = requests.get("https://www.google.com", timeout=5)
        return True
    except requests.ConnectionError:
        return False

In [50]:
def adad_farsi_to_english(inp: str):
    dc = {'۰': '0', '۱': '1', '۲': '2', '۳': '3', '۴': '4', '۵': '5', '۶': '6', '۷': '7', '۸': '8', '۹': '9'}
    out = ''
    for i in inp:
        if i in dc.keys():
            out += dc[i]
        else:
            out += ''
    if out.isdecimal():
        return int(out)
    else:
        return None

In [51]:
def get_tokens(inp_url, inp_json):
    out_tokens = []
    resp = requests.post(url=inp_url, json=inp_json)
    if resp.status_code == 200:
        data_v = resp.json()
        for j in data_v["web_widgets"]["post_list"]:
            try:
                out_tokens.append(j['data']['token'])
            except:
                continue
    else:
        return 0
    return out_tokens, data_v['last_post_date'], data_v['first_post_date']

In [52]:
def first_get_tokens(inp_url):
    f_post_date = 0
    #l_post_date = 0
    js_v = {"page": 0,
            "json_schema": {"category": {"value": "plot-old"}, "query": "زمین",
                            "sort": {"value": "sort_date"}},
            "last-post-date": int(str(time.time()).replace('.', ''))}

    try:
        l_p_date = requests.post(url=inp_url, json=js_v).json()['last_post_date']
    except:
        l_p_date = int(str(time.time()).replace('.', ''))
    out_tokens = []
    k = 0
    while True:
        json_v = {"page": k,
                  "json_schema": {"category": {"value": "plot-old"}, "query": "زمین",
                                  "sort": {"value": "sort_date"}},
                  "last-post-date": l_p_date}

        result = get_tokens(inp_url, json_v)
        if result == 0:
            break
        else:
            if k == 0:
                f_post_date = result[2]
            out_tokens += result[0]
            l_p_date = result[1]
            k += 1
    return out_tokens, l_p_date, f_post_date

In [53]:
def second_get_tokens(inp_url, end_date):
    f_post_date = 0
    out_tokens = []
    json_v = {"page": 0,
              "json_schema": {"category": {"value": "plot-old"}, "query": "زمین",
                              "sort": {"value": "sort_date"}},
              "last-post-date": int(str(time.time()).replace('.', ''))}
    try:
        l_p_date = requests.post(url=inp_url, json=json_v).json()['last_post_date']
    except:
        l_p_date = int(str(time.time()).replace('.', ''))
    k = 0
    while True:
        if l_p_date > end_date:
            js_v = {"page": k,
                    "json_schema": {"category": {"value": "plot-old"}, "query": "زمین",
                                    "sort": {"value": "sort_date"}},
                    "last-post-date": l_p_date}
            result = get_tokens(inp_url, js_v)
            if result != 0:
                if k == 0:
                    f_post_date = result[2]
                out_tokens += result[0]
                l_p_date = result[1]
            else:
                break
        else:
            break
    return out_tokens, l_p_date, f_post_date

In [54]:
def get_base_url(inp="https://api.divar.ir/v8/web-search/30/plot-old"):
    inp2 = input('Enter your url :')
    if inp2 == '':
        return inp
    else:
        return inp2


def get_base_url2(inp=None):
    if inp:
        return inp
    else:
        return "https://api.divar.ir/v8/web-search/30/plot-old"


def get_last_post_data(inp=None):
    if inp:
        return inp
    else:
        return int(str(time.time()).replace('.', ''))

In [55]:
def true_get_data2(inp_url, try_num, timeout):
    #o = string.ascii_letters + string.punctuation
    #c = 0
    make_url = inp_url
    for i in range(try_num):
        time.sleep(timeout)
        #print('-', end='')
        resp = requests.get(make_url)
        #make_url = make_url.replace('-', o[randint(0, len(o) - 1)])
        if resp.status_code == 200:
            page_ = resp.text
            if 'متر' in page_ and 'زمین' in page_ and 'متراژ' in page_ and 'قیمت کل' in page_ and 'قیمت هر متر' in page_:
                return resp
            else:
                continue
        else:
            continue
    return False

In [56]:
def extract_json_data(inp_url,try_num,timeout):
    true_req = true_get_data2(inp_url, try_num, timeout)
    if true_req is not False:
        out_data=[]
        page_json= true_req.json()
        for i in page_json['sections']:
            if i['section_name'] == 'TITLE':
                out_data.append(i['widgets'][0]['data']['title'])
            if i['section_name']=='LIST_DATA':
                out_data.append(adad_farsi_to_english(i['widgets'][0]['data']['value']))
                out_data.append(adad_farsi_to_english(i['widgets'][1]['data']['value']))
            if i['section_name'] == 'DESCRIPTION':
                out_data.append(i['widgets'][1]['data']['text'])
        if len(out_data)==4:
            return out_data
        else:
            return False
    else:
        return False

In [57]:
def get_data_by_tokens(inp_tokens,try_num,timeout):
    df_out=pd.DataFrame({'address':[],'description':[],'area':[],'price':[]})
    for i in tqdm(inp_tokens):
        dest_url=f"https://api.divar.ir/v8/posts-v2/web/{i}"
        dt=extract_json_data(dest_url,try_num,timeout)
        if dt is not False:
            df_out.loc[len(df_out.index)+1]=dt
    return df_out

In [58]:
def get_data_by_tokens2(inp_tokens,try_num,timeout):
    con_db=sqlite3.connect('real_state.db')
    %sql sqlite:///real_state.db
    %sql CREATE TABLE IF NOT EXISTS out_data (Address TEXT,Description TEXT,Area INTEGER,Price INTEGER)
    #df_out=pd.DataFrame({'address':[],'description':[],'area':[],'price':[]})
    for i in tqdm(inp_tokens):
        dest_url=f"https://api.divar.ir/v8/posts-v2/web/{i}"
        dt=extract_json_data(dest_url,try_num,timeout)
        if dt is not False:
            %sql INSERT INTO out_data(Address,Description,Area,Price) VALUES (:dt[0],:dt[1],:dt[2],:dt[3]))
    con_db.close()

In [59]:
def create_log():
    if 'log.txt' in os.listdir():
        with open('log.txt', 'r') as f:
            if f.read() == '':
                f.close()
                with open('log.txt', 'w+') as f:
                    f.write(f'{0},{time.time()}\n')
    else:
        with open('log.txt', 'w') as f:
            f.write(f'{0},{time.time()}\n')

In [60]:
create_log()

In [61]:
if internet_connection():
    try_num = 2
    timeout = 1
    with open('log.txt', 'r') as f:
        msg = f.readlines()[-1].split(',')
    if msg[0] == '0':
        tokens = first_get_tokens(get_base_url2())
        with open('log.txt', 'a') as f:
            f.write(f'{1},{time.time()},{tokens[1]},{tokens[2]}\n')
        get_data_by_tokens2(tokens[0], try_num, timeout)
    else:
        finish_date=int(msg[-1].replace('\n',''))
        tokens = second_get_tokens(get_base_url(),end_date=finish_date)
        with open('log.txt', 'a') as f:
            f.write(f'{int(msg[0]) + 1},{time.time()},{tokens[1]},{tokens[2]}\n')
        get_data_by_tokens2(tokens[0], try_num, timeout)

 * sqlite:///real_state.db
Done.


  0%|          | 1/1553 [00:01<32:03,  1.24s/it]

 * sqlite:///real_state.db
(sqlite3.OperationalError) near "[0]": syntax error
[SQL: INSERT INTO out_data(Address,Description,Area,Price) VALUES (?[0],?[1],?[2],?[3]))]
[parameters: (['زمین قولنامه ای موقعیت مکانی', 'بلوار امام رضا کوچه بعداز پمپ بنزین؛بعداز چهارراه انبار فاطمی\nتیر برق پا زمین \nدیوارکشی شده ؛سقف آهنی؛استخر ۳×۴\nآینده دار و ساخت ساز شده \nتخفیف پای معامله\nمعاوضه با۲۰۷هم دارم', 165, 720000000], ['زمین قولنامه ای موقعیت مکانی', 'بلوار امام رضا کوچه بعداز پمپ بنزین؛بعداز چهارراه انبار فاطمی\nتیر برق پا زمین \nدیوارکشی شده ؛سقف آهنی؛استخر ۳×۴\nآینده دار و ساخت ساز شده \nتخفیف پای معامله\nمعاوضه با۲۰۷هم دارم', 165, 720000000], ['زمین قولنامه ای موقعیت مکانی', 'بلوار امام رضا کوچه بعداز پمپ بنزین؛بعداز چهارراه انبار فاطمی\nتیر برق پا زمین \nدیوارکشی شده ؛سقف آهنی؛استخر ۳×۴\nآینده دار و ساخت ساز شده \nتخفیف پای معامله\nمعاوضه با۲۰۷هم دارم', 165, 720000000], ['زمین قولنامه ای موقعیت مکانی', 'بلوار امام رضا کوچه بعداز پمپ بنزین؛بعداز چهارراه انبار فاطمی\nتیر برق پا زمین \nدیو

  0%|          | 2/1553 [00:02<32:36,  1.26s/it]

 * sqlite:///real_state.db
(sqlite3.OperationalError) near "[0]": syntax error
[SQL: INSERT INTO out_data(Address,Description,Area,Price) VALUES (?[0],?[1],?[2],?[3]))]
[parameters: (['زمین قابل ساخت داخل بافت', 'زمین مسکونی روستای ارمک نزدیک امام زاده سند شورا بعد از ساخت سند ششدانگ میدن', 150, 450000000], ['زمین قابل ساخت داخل بافت', 'زمین مسکونی روستای ارمک نزدیک امام زاده سند شورا بعد از ساخت سند ششدانگ میدن', 150, 450000000], ['زمین قابل ساخت داخل بافت', 'زمین مسکونی روستای ارمک نزدیک امام زاده سند شورا بعد از ساخت سند ششدانگ میدن', 150, 450000000], ['زمین قابل ساخت داخل بافت', 'زمین مسکونی روستای ارمک نزدیک امام زاده سند شورا بعد از ساخت سند ششدانگ میدن', 150, 450000000])]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


...

 * sqlite:///real_state.db
(sqlite3.OperationalError) near "[0]": syntax error
[SQL: INSERT INTO out_data(Address,Description,Area,Price) VALUES (?[0],?[1],?[2],?[3]))]
[parameters: (['زمین راومد کوچه شهید عیوضی', 'سلام مساحت زمین ۱۶۵ متر منطقه مسکونی\nکوچه ده متری\nتمام امتیازها پای زمین\nقولنامه از ریشه و معتبر\nآدرس راوند \nکوچه شهید عیوضی\nکوچه سعدی (باغ رضا)\nاطراف در حال ساخت میباشد\nتشکر از عوامل ومدیر دیوار', 165, 430000000], ['زمین راومد کوچه شهید عیوضی', 'سلام مساحت زمین ۱۶۵ متر منطقه مسکونی\nکوچه ده متری\nتمام امتیازها پای زمین\nقولنامه از ریشه و معتبر\nآدرس راوند \nکوچه شهید عیوضی\nکوچه سعدی (باغ رضا)\nاطراف در حال ساخت میباشد\nتشکر از عوامل ومدیر دیوار', 165, 430000000], ['زمین راومد کوچه شهید عیوضی', 'سلام مساحت زمین ۱۶۵ متر منطقه مسکونی\nکوچه ده متری\nتمام امتیازها پای زمین\nقولنامه از ریشه و معتبر\nآدرس راوند \nکوچه شهید عیوضی\nکوچه سعدی (باغ رضا)\nاطراف در حال ساخت میباشد\nتشکر از عوامل ومدیر دیوار', 165, 430000000], ['زمین راومد کوچه شهید عیوضی', 'سلام مساحت زمین ۱۶۵ 

  2%|▏         | 25/1553 [00:32<35:44,  1.40s/it]

 * sqlite:///real_state.db
(sqlite3.OperationalError) near "[0]": syntax error
[SQL: INSERT INTO out_data(Address,Description,Area,Price) VALUES (?[0],?[1],?[2],?[3]))]
[parameters: (['زمین ۱۲۰ متری بلوار امام رضا کوچه عمارت فخریه', 'فرصتی استثنایی \nمعاوضه با خودرو و آپارتمان\nنقد با تخفیف ویژه و شرایط اقساط \nفقط و فقط تا پایان ر ... (209 characters truncated) ... ایتا به همین خط و ارسال موقعیت مکانی توسط من\nقولنامه معتبر دست اول از رعیت منطقه\nاولین بازدید=پسندیدن= خرید \nتماس بگیرید۰\nیا علی', 120, 350000000], ['زمین ۱۲۰ متری بلوار امام رضا کوچه عمارت فخریه', 'فرصتی استثنایی \nمعاوضه با خودرو و آپارتمان\nنقد با تخفیف ویژه و شرایط اقساط \nفقط و فقط تا پایان ر ... (209 characters truncated) ... ایتا به همین خط و ارسال موقعیت مکانی توسط من\nقولنامه معتبر دست اول از رعیت منطقه\nاولین بازدید=پسندیدن= خرید \nتماس بگیرید۰\nیا علی', 120, 350000000], ['زمین ۱۲۰ متری بلوار امام رضا کوچه عمارت فخریه', 'فرصتی استثنایی \nمعاوضه با خودرو و آپارتمان\nنقد با تخفیف ویژه و شرایط اقساط \nفقط و فقط ت

  2%|▏         | 25/1553 [00:33<33:55,  1.33s/it]


KeyboardInterrupt: 

In [64]:
with open('log.txt' ,'r') as f:
    ms=(f.readlines())

In [66]:
ms[-1]

'1,1706872232.662624,-1,1706801518422228\n'

In [24]:
with open('test.txt','r') as f:
    ms=f.readline()

In [28]:
ms.split('-')[-1].replace('\n','')

'1706813896.9439313'